# Day 5 – Error Handling in LLM Applications

This notebook demonstrates practical error-handling techniques used in LLM-based systems.

## 1. Input Validation Error Handling

In [1]:

def validate_prompt(prompt: str, max_chars: int = 4000):
    if not isinstance(prompt, str):
        raise TypeError("Prompt must be a string")
    if not prompt.strip():
        raise ValueError("Prompt cannot be empty")
    if len(prompt) > max_chars:
        raise ValueError("Prompt exceeds maximum allowed length")
    return True

# Example
validate_prompt("This is a valid prompt")


True

## 2. Token / Context Window Error Handling (Chunking)

In [2]:

def chunk_text(text, chunk_size=800):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

long_text = "word " * 3000
chunks = chunk_text(long_text)
len(chunks)


4

## 3. API-Level Error Handling with Retry & Backoff

In [3]:

import time

def call_llm_api(api_call, retries=3):
    for attempt in range(retries):
        try:
            return api_call()
        except Exception as e:
            if "429" in str(e) or "timeout" in str(e).lower():
                time.sleep(2 ** attempt)
            else:
                raise e
    raise Exception("LLM API failed after retries")

# Mock API
def mock_api():
    raise Exception("429 Rate limit")

# call_llm_api(mock_api)


## 4. Authentication Error Handling

In [4]:

import os

def load_api_key():
    api_key = os.getenv("LLM_API_KEY")
    if not api_key:
        raise EnvironmentError("API key not found")
    return api_key


## 5. Model Output Error Handling (JSON Validation)

In [5]:

import json

def parse_llm_response(response_text):
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON response from LLM")

# Example
parse_llm_response('{"status": "ok"}')


{'status': 'ok'}

## 6. Partial / Truncated Response Handling

In [6]:

def is_response_complete(response):
    return response.strip().endswith("}")

response = '{"data": "incomplete"'
is_response_complete(response)


False

## 7. Network & Timeout Error Handling

In [7]:

import requests

try:
    requests.get("https://example.com", timeout=0.0001)
except requests.exceptions.Timeout:
    print("Request timed out")


Request timed out


## 8. Fallback & Graceful Degradation

In [8]:

def get_llm_response(prompt):
    try:
        raise Exception("LLM Down")
    except Exception:
        return "Service temporarily unavailable"

get_llm_response("hello")


'Service temporarily unavailable'

## 9. Logging Errors Safely

In [9]:

import logging
logging.basicConfig(level=logging.ERROR)

def log_error(error, request_id):
    logging.error(f"Request {request_id} failed: {error}")

log_error("Timeout", "REQ001")


ERROR:root:Request REQ001 failed: Timeout


## 10. End-to-End Error Handling Flow

In [10]:

def process_prompt(prompt):
    try:
        validate_prompt(prompt)
        response = '{"result": "success"}'
        return parse_llm_response(response)
    except Exception as e:
        log_error(e, "REQ999")
        return {"error": "Unable to process request"}

process_prompt("Test prompt")


{'result': 'success'}